In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("database.sqlite")

In [3]:
master = pd.read_sql("select * from Emails e left join EmailReceivers r on e.Id = r.EmailId where r.PersonId=80", con)

In [4]:
master.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText,Id,EmailId,PersonId
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,,UNCLASSIFIED\nU.S. Department of State\nCase N...,1,1,80
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,,,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...,2,2,80
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,"Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...,4,3,80
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,"Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,,UNCLASSIFIED\nU.S. Department of State\nCase N...,5,4,80
4,7,C05739560,"ANTI-MUSLIM FILM DIRECTOR IN HIDING, FOLLOWING...",H,"Mills, Cheryl D",32,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739560...,F-2015-04841,...,"Wednesday, September 12, 2012 4:00 PM",F-2015-04841,C05739560,05/13/2015,RELEASE IN FULL,,UNCLASSIFIED\nU.S. Department of State\nCase N...,8,7,80


In [5]:
df_annotated = pd.read_csv('project-16038-at-2022-09-30-14-36-294aef1d.csv')

In [6]:
len(df_annotated)

1139

In [7]:
df_annotated.head()

,dbId,text,docNumber,id,sentiment,annotator,annotation_id,created_at,updated_at,lead_time,agreement
0,10,<b>Subject:</b> <i>MORE ON LIBYA</i><br/><b>Bo...,C05739563,26842916,Schedule,amir.nagri@gmail.com,10584767,2022-09-30T14:34:56.928113Z,2022-09-30T14:34:56.928128Z,8.167,100.0
1,16,<b>Subject:</b> <i>THANK YOU</i><br/><b>Body</...,C05739571,26842922,Schedule,amir.nagri@gmail.com,10584435,2022-09-30T14:26:07.001964Z,2022-09-30T14:26:07.001979Z,3.070,100.0
2,28,<b>Subject:</b> <i>AMB STEVENS </i><br/><b>Bod...,C05739589,26842929,Schedule,amir.nagri@gmail.com,10582479,2022-09-30T12:56:29.517365Z,2022-09-30T12:56:29.517381Z,5.517,100.0
3,41,<b>Subject:</b> <i>LIBYAN</i><br/><b>Body</b>:...,C05739608,26842940,Ignore,amir.nagri@gmail.com,10582685,2022-09-30T13:08:22.832607Z,2022-09-30T13:08:22.832624Z,2.426,100.0
4,47,<b>Subject:</b> <i>H: ROMNEY'S LAST GAMBIT. GO...,C05739615,26842943,Ignore,amir.nagri@gmail.com,10584076,2022-09-30T14:17:10.974393Z,2022-09-30T14:17:10.974410Z,1.307,100.0


In [8]:
df_annotated.drop_duplicates(subset=['docNumber'], inplace=True, keep='last')

In [9]:
# drop rows with sentiment ignore
df_annotated.drop(df_annotated[df_annotated['sentiment'] == 'Ignore'].index, inplace=True)

In [10]:
len(df_annotated)

992

In [11]:
df_with_annot = master.join(df_annotated.set_index('docNumber'), on='DocNumber')

In [12]:
# drop rows that are not annotated
df_annot_rows = df_with_annot[df_with_annot['sentiment'].notna()].copy()

In [13]:
df_annot_rows.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,dbId,text,id,sentiment,annotator,annotation_id,created_at,updated_at,lead_time,agreement
6,10,C05739563,MORE ON LIBYA,H,,,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739563...,F-2015-04841,...,10.0,<b>Subject:</b> <i>MORE ON LIBYA</i><br/><b>Bo...,26842916.0,Schedule,amir.nagri@gmail.com,10584767.0,2022-09-30T14:34:56.928113Z,2022-09-30T14:34:56.928128Z,8.167,100.0
9,13,C05739567,HEY,H,"Sullivan, Jacob J",87,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739567...,F-2015-04841,...,13.0,<b>Subject:</b> <i>HEY</i><br/><b>Body</b>:<pr...,26842919.0,Delete,amir.nagri@gmail.com,10577804.0,2022-09-30T07:59:52.201247Z,2022-09-30T07:59:52.201262Z,3.605,100.0
12,16,C05739571,THANK YOU,H,"Koh, Harold Hongju",77,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739571...,F-2015-04841,...,16.0,<b>Subject:</b> <i>THANK YOU</i><br/><b>Body</...,26842922.0,Schedule,amir.nagri@gmail.com,10584435.0,2022-09-30T14:26:07.001964Z,2022-09-30T14:26:07.001979Z,3.070,100.0
19,28,C05739589,AMB STEVENS,H,"Mills, Cheryl D",32,2012-09-13T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739589...,F-2015-04841,...,28.0,<b>Subject:</b> <i>AMB STEVENS </i><br/><b>Bod...,26842929.0,Schedule,amir.nagri@gmail.com,10582479.0,2022-09-30T12:56:29.517365Z,2022-09-30T12:56:29.517381Z,5.517,100.0
38,55,C05739626,THANKS,H,"Burns, William J",216,2012-09-14T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739626...,F-2015-04841,...,55.0,<b>Subject:</b> <i>THANKS</i><br/><b>Body</b>:...,26842948.0,Delete,amir.nagri@gmail.com,10582866.0,2022-09-30T13:21:29.830406Z,2022-09-30T13:21:29.830423Z,2.048,100.0


In [14]:
# prepare fine tuning string
df_annot_rows['train_txt'] = df_annot_rows['MetadataSubject'].str.strip() + " <end> " + df_annot_rows['ExtractedBodyText'].str.strip()

In [15]:
df_annot_rows['train_txt'] = df_annot_rows['train_txt'].str.replace("\n", " <br/> ").str.replace("\t","  ")

In [16]:
# select columns to export
df_export = df_annot_rows[['train_txt', 'sentiment']].copy()

In [17]:
df_export['sentiment'].unique()

array(['Schedule', 'Delete', 'Delegate', 'Do'], dtype=object)

In [18]:
# update the sentiments to match email labels
df_export.loc[df_export['sentiment'] == 'Do', 'sentiment'] = 'Now'
df_export.loc[df_export['sentiment'] == 'Schedule', 'sentiment'] = 'Later'
df_export.loc[df_export['sentiment'] == 'Delete', 'sentiment'] = 'Discard'
df_export['sentiment'].unique()

array(['Later', 'Discard', 'Delegate', 'Now'], dtype=object)

In [19]:
# export csv to fine tune the model
df_export.to_csv('train_classify_finetune.csv', index=False, header=False)

In [20]:
!head train_classify_finetune.csv

"MORE ON LIBYA <end> B6 <br/> Wednesday, September 12, 2012 6:16 PM <br/> Fwd: more on libya <br/> Libya 37 sept 12 12.docx <br/> Sending direct. Just in. <br/> Sent from my Verizon Wireless 4G LTE DRUID",Later
HEY <end> Fyi,Discard
"THANK YOU <end> I asked to attend your svtc today with Embassy Tripoli, because had first met so many of that staff when I went with <br/> you from Malta to Tripoli for the reopening of our Embassy. <br/> Today's deaths hit me much harder than I would have guessed. I am always proud to serve under you, but never have <br/> your words been more meaningful than on today's svtc. Every day of your tenure has been extraordinary, but none <br/> more so than today. Thank you again for your inspirational leadership and example. <br/> As ever, <br/> Harold <br/> U.S. Department of State <br/> Case No. F-2015-04841 <br/> Doc No. C05739571 <br/> Date: 05/13/2015 <br/> STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. <br/> SUBJECT TO AGREEMENT ON SENSITIVE INFORM